# AugmentedGPT Playground

Please run `poetry install` under the project root directory. This will install all the necessary dependencies for you.

## Imports and set API key

In [1]:
from agentia import Agent, UserMessage, plugins
import dotenv
from typing import Annotated

dotenv.load_dotenv()

True

## Create a GPT instance that can use tools.

In [2]:
def send_email(
    topic: Annotated[str, "The email topic"],
    to: Annotated[str, "The recipient email address"],
    content: Annotated[str, "The content of the email"],
):
    """Send an email to a recipient"""
    print(f"--- SEND EMAIL ---")
    print(f"TOPIC: {topic}")
    print(f"TO: {to}")
    print(f"CONTENT:\n\n{content}")
    print(f"---")
    return "done."


agent = Agent(tools=[send_email, plugins.CalculatorPlugin()], debug=True)

response = await agent.chat_completion(
    [
        UserMessage(
            "Send Steve an email and tell him the distance between moon and earth divided by two."
        ),
    ]
)

print(response)

--- SEND EMAIL ---
TOPIC: Distance from Moon to Earth (Divided by 2)
TO: steve@example.com
CONTENT:

The distance from the Moon to the Earth is approximately 384,400 km. When divided by two, it is 192,200 km.
---
I have sent an email to Steve informing him that the distance between the Moon and Earth divided by two is 192,200 km.
